# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,3.22,88,48,1.98,GS,1698593041
1,1,vila velha,-20.3297,-40.2925,31.97,51,0,7.20,BR,1698592958
2,2,tiassale,5.8983,-4.8283,31.00,60,58,0.84,CI,1698593042
3,3,nouadhibou,20.9310,-17.0347,25.99,65,0,6.69,MR,1698593043
4,4,isafjordur,66.0755,-23.1240,1.02,80,0,0.00,IS,1698592810


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', # x-axis: Longitude
    'Lat', # y-axis: Latitude
    geo=True, # To ensure the data is plotted on a map
    tiles='OSM', # Using OpenStreetMap as the tile source
    size='Humidity', # Size of the point will be based on the humidity value
    hover_cols=['City', 'Humidity'], # On hover, display the city name and its humidity
    color='Humidity', # Color of the point will be based on the humidity value
    colormap='viridis', # Using 'viridis' colormap
    title='City Locations by Humidity'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 10) & (city_data_df["Max Temp"] <= 35) & 
    (city_data_df["Wind Speed"] < 20) & 
    (city_data_df["Cloudiness"] < 50) & 
    (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <= 80)
]


# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,vila velha,-20.3297,-40.2925,31.97,51,0,7.20,BR,1698592958
3,3,nouadhibou,20.9310,-17.0347,25.99,65,0,6.69,MR,1698593043
17,17,jagtial,18.8000,78.9333,22.50,59,0,0.51,IN,1698593047
25,25,buchanan,5.8808,-10.0467,29.01,71,15,3.69,LR,1698593049
27,27,hamamatsu,34.7000,137.7333,13.90,65,0,0.89,JP,1698593049


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,vila velha,BR,-20.3297,-40.2925,51,
3,nouadhibou,MR,20.9310,-17.0347,65,
17,jagtial,IN,18.8000,78.9333,59,
25,buchanan,LR,5.8808,-10.0467,71,
27,hamamatsu,JP,34.7000,137.7333,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 100
params = {
    "apiKey": geoapify_key,
    "types": "accommodation", 
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"circle:{lng},{lat},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vila velha - nearest hotel: No hotel found
nouadhibou - nearest hotel: No hotel found
jagtial - nearest hotel: No hotel found
buchanan - nearest hotel: No hotel found
hamamatsu - nearest hotel: No hotel found
pingliang - nearest hotel: No hotel found
alghero - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
tarfaya - nearest hotel: No hotel found
birao - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
al minya - nearest hotel: No hotel found
korla - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
sabzevar - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
camana - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
e

,City,Country,Lat,Lng,Humidity,Hotel Name
1,vila velha,BR,-20.3297,-40.2925,51,No hotel found
3,nouadhibou,MR,20.9310,-17.0347,65,No hotel found
17,jagtial,IN,18.8000,78.9333,59,No hotel found
25,buchanan,LR,5.8808,-10.0467,71,No hotel found
27,hamamatsu,JP,34.7000,137.7333,65,No hotel found
...,...,...,...,...,...,...
556,tapes,BR,-30.6733,-51.3958,69,No hotel found
563,sultanah,SA,24.4926,39.5857,53,No hotel found
564,korhogo,CI,9.4500,-5.6333,28,No hotel found
566,datong,CN,40.0936,113.2914,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    hover_cols=["Hotel Name", "Country"], 
    size="Humidity",  # Setting the size based on humidity, as per the previous steps
    color="Humidity", 
    colormap="viridis", 
    title="Hotel Locations based on Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    height=500,
    width=800
)

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,Hotel Name,Country)